In [1]:
import openai
import pandas as pd
import json
from IPython.display import Markdown
import sys
from tqdm import tqdm
import pyperclip
import yaml

# Load prompt template

In [2]:
sys.path.append('..')
from utils import *
from functools import partial

In [3]:
prompt_dict = partial(load_prompt_template,file='../prompt_template.md')

In [4]:
def drop_text_after(string):
    string_new=''
    for line in string.split('\n'):
        if '===' in line:
            break
        else:
            if 'You should recall that' not in line:
                string_new='\n'.join([string_new,line])
    return string_new[1:]


In [ ]:
with open('1010.1819.jsonl','r') as f:
    kwargs_jsonl= [json.loads(line) for line in f]

In [6]:
with open('1010.1819.yaml','r') as f:
    kwargs_jsonl= yaml.safe_load(f)

In [7]:
def prompt_template(descriptor):
    return drop_text_after(prompt_dict()[descriptor['task']])

# Load automatically filled prompt

In [8]:
def parse_markdown_to_dict(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    result_dict = {}
    current_title = None
    content_buffer = []

    for line in lines:
        stripped_line = line.strip()

        if stripped_line.startswith("# "):
            if current_title:  # If there's already a title detected
                result_dict[current_title] = '\n'.join(content_buffer).strip()
                content_buffer = []
            current_title = stripped_line[2:]
        else:
            content_buffer.append(line)

    # For the last title-content pair
    if current_title and content_buffer:
        result_dict[current_title] = '\n'.join(content_buffer).strip()

    return result_dict

In [9]:
filename = "1010.1819_extractor.md"
filled_dict = parse_markdown_to_dict(filename)


# Return place holder

In [15]:
import re

def extract_filled_values(template_str):
    template_str = template_str.replace('{{', '').replace('}}', '')
    # Extract placeholders from the template
    # placeholders = re.findall(r"\{(\w+)\}", template_str)
    placeholders = re.findall(r"[\{]([\w\|\-$ ,.\{\}]+?)[\}]", template_str)
    placeholders_optional = re.findall(r"[\[]([\w\|\-$ ,.\{\}]+)[\]]", template_str)
    # Create a regex pattern to match the filled values based on the placeholders
    placeholders_unique=[]
    for placeholder in placeholders+placeholders_optional:
        if placeholder not in placeholders_unique:
            placeholders_unique.append(placeholder)
    return placeholders_unique
    
    # pattern = re.sub(r"\{(\w+)\}", r"(?P<\1>\\S+)", template_str)

    # # Find the values using the generated pattern
    # match = re.match(pattern, filled_str)
    # if match:
    #     return {placeholder: match.group(placeholder) for placeholder in placeholders}
    # else:
    #     return None



In [41]:
# Example
template_str = drop_text_after(prompt_dict()["Construct full Hamiltonian after HF"])
# filled_str = filled_dict['Construct Kinetic Hamiltonian (continuum version, single-particle)']
results = extract_filled_values(template_str)
print(results)

['Hartree_Fock_symbol', 'kinetic_symbol', 'expression_kinetic', 'int_symbol', 'expression_int', 'Ham_symbol', 'definition_of_variables']


In [42]:
string= 'placeholder:\n'
for result in results:
    string+=f'    {result}:\n'
    string+='      LLM: \n'
    string+='      human: \n'
    string+='      score: \n'
    string+='        Haining: \n'
    string+='        Will: \n'
    string+='        Subha: \n'
print(string)
pyperclip.copy(string)    

placeholder:
    Hartree_Fock_symbol:
      LLM: 
      human: 
      score: 
        Haining: 
        Will: 
        Subha: 
    kinetic_symbol:
      LLM: 
      human: 
      score: 
        Haining: 
        Will: 
        Subha: 
    expression_kinetic:
      LLM: 
      human: 
      score: 
        Haining: 
        Will: 
        Subha: 
    int_symbol:
      LLM: 
      human: 
      score: 
        Haining: 
        Will: 
        Subha: 
    expression_int:
      LLM: 
      human: 
      score: 
        Haining: 
        Will: 
        Subha: 
    Ham_symbol:
      LLM: 
      human: 
      score: 
        Haining: 
        Will: 
        Subha: 
    definition_of_variables:
      LLM: 
      human: 
      score: 
        Haining: 
        Will: 
        Subha: 



# Validate yaml

1. Verify score is not None
2. Verify if score<2, human is not None; if score == 2, human is None and LLM is not None
3. Verify not (LLM and human are both None)

In [12]:
import yaml
with open('1010.1819.yaml','r') as f:
    kwargs_yaml = yaml.safe_load(f)

In [13]:
string=''
for kwargs in kwargs_jsonl:
    string+=f"# {kwargs['task']}\n"
    string+=drop_text_after(prompt_dict()[kwargs['task']])

with open('../cmp2.md','w') as f:
    f.write(string)